In [ ]:
%matplotlib qt
import numpy as np
import hyperspy.api as hs
from ncempy.io import dm
import matplotlib.pyplot as plt
from scipy.signal import peak_widths, find_peaks
import math

In [ ]:
#convert text files
def txtconverter(numpy_array):
    file = str(numpy_array).replace('[','')
    file = file.replace(']','')
    data = np.fromstring(file, sep=',')
    return data

#sorting data into counts and eV
def find_counts(data):
    counts = data[1:-1:2]
    return counts

def find_ev(data):
    ev = data[0:-1:2]
    return ev

#plot the spectrum with HyperSpy
def hyperspy_plot(ev, counts):
    s = hs.signals.EELSSpectrum(counts)
    s.axes_manager[0].scale = np.diff(ev).mean()
    s.axes_manager[0].unit = 'eV'
    s.axes_manager[0].offset = ev[0]
    s.axes_manager[0].name = 'Energy'
    return s

#FWHM comparisons
def FWHM_testing(alpha, gamma, hs_signal, hs_deconvolved, height):
    
    peaks1, _ = find_peaks(hs_signal, height=1)
    results_half_signal = peak_widths(hs_signal, peaks1, rel_height=0.5)
    
    peaks2, _ = find_peaks(hs_deconvolved, height=height)
    results_half_deconvolved = peak_widths(hs_deconvolved, peaks2, rel_height=0.5)
    
    FWHM_signal = 4 / 1000 * results_half_signal[0]
    FWHM_deconvolved = 4 / 1000 * results_half_deconvolved[0]
    
    Lorentzian_FWHM = 2 * gamma
    Gaussian_FWHM = 2 * alpha
    
    relative_error = abs((FWHM_deconvolved[0] - Lorentzian_FWHM)/Lorentzian_FWHM*100)
    
    print("FWHM of signal =", FWHM_signal[0], "eV", 
          "\nFWHM of deconvolved =", FWHM_deconvolved[0], "eV", 
          "\nFWHM of Lorentzian =", Lorentzian_FWHM, "eV", 
          "\nRelative error =",  math.trunc(relative_error), "%\n")
    
#plotting the noise
def noise(Spectrum, deconvolved, PSF):
    noise = np.subtract(Spectrum, np.convolve(deconvolved, PSF, mode='same'))
    return noise

#Richardson-Lucy algorithm (code from Edson Bellido)
def RL(iterations, PSF, Spectrum):
    RL4 = np.copy(Spectrum)
    for i in range(iterations):
        RL1 = np.convolve(PSF, RL4, mode='same')
        RL2 = np.divide(Spectrum,RL1)
        RL3 = np.convolve(PSF, RL2, mode='same')
        RL4 = np.multiply(RL3, RL4)
    return RL4

MEM algorithm from Meinel 1986 article, equation 71. Derived from the Gaussian noise process but states that it is similar to algorithms derived from the Poisson noise process. Also states that squaring in the convolution makes the noise propogation worse than the Poisson noise algorithms.

In [ ]:
def MEM(iterations, PSF, Spectrum):
    MEM = np.copy(Spectrum)
    for i in range(iterations):
        MEM1 = np.convolve(PSF, MEM, mode='same')
        MEM2 = np.divide(Spectrum, MEM1)
        MEM3 = np.multiply(MEM2, MEM2)
        MEM4 = np.convolve(PSF, MEM3, mode='same')
        MEM5 = np.multiply(MEM4, MEM)
        MEM = MEM5
    return MEM

In [ ]:
#load file as numpy array
Signal = np.loadtxt("D:\Downloads\Signal1.txt",dtype="str")
PSF = np.loadtxt("D:\Downloads\PSF1.txt", dtype='str')
Real = np.loadtxt("D:\Downloads\Real1.txt", dtype='str')

#convert text file to usable numpy array
signal = txtconverter(Signal)
psf = txtconverter(PSF)
real = txtconverter(Real)

#separate data into counts and ev
signal_counts = find_counts(signal)
psf_counts = find_counts(psf)
real_counts = find_counts(real)
ev = find_ev(signal)

In [ ]:
MEM_deconvolve = MEM(150, psf_counts, signal_counts)
s_MEM = hyperspy_plot(ev, MEM_deconvolve)

RL_deconvolve = RL(150, psf_counts, signal_counts)
s_RL = hyperspy_plot(ev, RL_deconvolve)

s_signal = hyperspy_plot(ev, signal_counts)

In [ ]:
print("MEM")
FWHM_MEM = FWHM_testing(0.1, 0.1, s_signal, s_MEM, 0.5)

print("RL")
FWHM_RL = FWHM_testing(0.1, 0.1, s_signal, s_RL, 1)

In [ ]:
s_MEM.plot()

In [ ]:
s_RL.plot()

In [ ]:
s_signal.plot()

In [ ]:
noise_MEM = noise(signal_counts, MEM_deconvolve, psf_counts)
s_noise_MEM = hyperspy_plot(ev, noise_MEM)

noise_RL = noise(signal_counts, RL_deconvolve, psf_counts)
s_noise_RL = hyperspy_plot(ev, noise_RL)

In [ ]:
s_noise_MEM.plot()

In [ ]:
s_noise_RL.plot()